In [1]:
!pip install yfinance
!pip install pandas
!pip install plotly

In [2]:
import plotly.graph_objects as go
import yfinance as yf
import pandas as pd
import datetime
import numpy as np
from sklearn.linear_model import LinearRegression

In [3]:
def get_stock_data(ticker_symbol, start_date, gap_days):
  """
  Fetches historical stock data for a given ticker from a start date to today.

  Args:
    ticker_symbol (str): The ticker symbol for the stock.
    start_date (str): The start date in 'YYYY-MM-DD' format.

  Returns:
    pd.DataFrame: A DataFrame containing the historical 'Open' and 'Close'
                  prices, a 'day' column, and log2 transformations,
                  or an empty DataFrame if no data is found or an error occurs.
  """
  end_date = datetime.date.today().strftime("%Y-%m-%d")
  try:
    ticker = yf.Ticker(ticker_symbol)
    hist = ticker.history(start=start_date, end=end_date)

    if not hist.empty:
      hist = hist[['Open', 'Close']]
      hist.columns = [f'{ticker_symbol}_Open', f'{ticker_symbol}_Close']
      hist['day'] = range(1, len(hist) + 1) # Start 'day' from 1
      hist['day'] = hist['day'] + gap_days # Offset the day count by 20 as in previous code

      # Apply log2 transformation
      hist['day_log2'] = np.log2(hist['day'])
      hist[f'{ticker_symbol}_Open_log2'] = np.log2(hist[f'{ticker_symbol}_Open'])
      hist[f'{ticker_symbol}_Close_log2'] = np.log2(hist[f'{ticker_symbol}_Close'])

      # Fit the linear regression model
      model = LinearRegression()
      X = hist[['day_log2']].values
      y = hist[f'{ticker_symbol}_Close_log2'].values
      model.fit(X, y)

      # Get intercept and slope
      intercept = model.intercept_
      slope = model.coef_

      print(f"Intercept: {intercept}")
      print(f"Slope: {slope}")

      # Create the 'log2_price_prediction' column
      hist['log2_price_prediction'] = intercept + slope * hist['day_log2']
      hist['price_prediction'] = np.exp2(hist['log2_price_prediction'])


      return hist
    else:
      print(f"No data found for {ticker_symbol} from {start_date} to {end_date}")
      return pd.DataFrame()

  except Exception as e:
    print(f"Could not fetch data for {ticker_symbol}: {e}")
    return pd.DataFrame()

def plot_stock_data(df, ticker_symbol):
  """
  Plots log2 open price vs log2 day and log2 open price vs date using Plotly.

  Args:
    df (pd.DataFrame): The DataFrame containing the stock data with
                       'day_log2', 'SNEK25264-USD_Open_log2', and index as date.
    ticker_symbol (str): The ticker symbol used in the DataFrame columns.
  """
  if df.empty:
    print("DataFrame is empty, cannot plot.")
    return

  # Plot log2 open price vs log2 day
  trace_open_day = go.Scatter(
      x=df['day_log2'],
      y=df[f'{ticker_symbol}_Open_log2'],
      mode='lines',
      name='Log2 Open Price vs Log2 Day',
      line=dict(color='red')
  )
  trace_pred_day = go.Scatter(
      x=df['day_log2'],
      y=df['log2_price_prediction'],
      mode='lines',
      name='Price_Pred vs Log2 Day',
      line=dict(color='blue')
  )

  layout_log2_day = go.Layout(
      title=f'{ticker_symbol} Log2 Open Price vs Log2 Day',
      xaxis=dict(title='Log2 Day'),
      yaxis=dict(title='Log2 Open Price')
  )

  fig_log2_day = go.Figure(data=[trace_open_day, trace_pred_day], layout=layout_log2_day)
  fig_log2_day.show()

  # Plot log2 open price vs date
  trace_open_date = go.Scatter(
      x=df.index, # Use the DataFrame index for dates
      y=df[f'{ticker_symbol}_Open_log2'],
      mode='lines',
      name='Log2 Open Price vs Date',
      line=dict(color='red')
  )

  trace_pred_day = go.Scatter(
      x=df.index,
      y=df['log2_price_prediction'],
      mode='lines',
      name='Price_Pred vs Log2 Date',
      line=dict(color='blue')
  )

  layout_date = go.Layout(
      title=f'{ticker_symbol} Log2 Open Price vs Date',
      xaxis=dict(title='Date'),
      yaxis=dict(title='Log2 Open Price')
  )

  fig_date = go.Figure(data=[trace_open_date, trace_pred_day], layout=layout_date)
  fig_date.show()

# Example usage of the functions:
# Replace with your desired ticker and start date
# ticker_to_fetch = "SNEK25264-USD"
# start_date_to_fetch = "2023-04-28"
#
# stock_df = get_stock_data(ticker_to_fetch, start_date_to_fetch)
#
# if not stock_df.empty:
#   plot_stock_data(stock_df, ticker_to_fetch)


In [4]:
df = get_stock_data("SNEK25264-USD", "2023-04-28", gap_days=20)
plot_stock_data(df, "SNEK25264-USD")

Intercept: -17.86294336579659
Slope: [0.9640798]


In [5]:
df = get_stock_data("ADA-USD", "2017-09-23", gap_days=50)
plot_stock_data(df, "ADA-USD")

Intercept: -8.966988472632655
Slope: [0.69963839]


In [6]:
ticker='ETH-USD'
start_date='2017-11-09'
gap_days=800
df = get_stock_data(ticker, start_date, gap_days)
plot_stock_data(df, ticker)

Intercept: -12.662093351290324
Slope: [2.05217253]
